In [1]:
from requests import get
from bs4 import BeautifulSoup
import os
import datetime
import smtplib
from email.mime.multipart import MIMEMultipart
from email.mime.text import MIMEText
import smtplib
import urllib.request
from PIL import Image
import sys
sys.getdefaultencoding()
import json
import time
import pandas as pd

In [2]:
url_base = 'https://www.worldometers.info/coronavirus/'
urls = [url_base+'country/spain/',url_base+'country/italy/',url_base]
countries = ['España:','Italia:', 'todo el Mundo:']
scrapped_info = ['Infectados','Fallecidos','Recuperados']
result=""
for url in urls:
    result+="Afectados en "+countries[urls.index(url)]+os.linesep
    html_soup = BeautifulSoup(get(url).text, 'html.parser')
    virus_containers = html_soup.find_all('div', class_ = 'maincounter-number')
    for number in virus_containers:
        result+=scrapped_info[virus_containers.index(number)]+": "+number.find('span').get_text()+os.linesep
    result+=os.linesep
result+=os.linesep

In [3]:
def extractMapImage():
    return Image.open(urllib.request.urlopen("https://covid19.isciii.es/resources/Mapa.jpg"))
def extractCurveStatusImage():
    return Image.open(urllib.request.urlopen('https://covid19.isciii.es/resources/CURVASTATUS.png')).convert('RGB')

In [71]:
def getRegion(region):
    now = datetime.datetime.now()
    date_str = str(now.year).zfill(2)+'-'+str(now.month).zfill(2)+'-'+str(now.day-1).zfill(2)
    api_url_base = 'https://api.covid19tracking.narrativa.com/api/'+date_str+'/country/spain/region/'+region
    time.sleep(3)
    sevilla = []
    headers = {'Content-Type': 'application/json','Authorization': 'Bearer {0}'}
    response = get(api_url_base, headers=headers)
    if response.status_code == 200:
        json_region = json.loads(response.content.decode('utf-8'))['dates'][date_str]['countries']['Spain']['regions'][0]

        result = ['Nuevos infectados: '+str(json_region['today_new_confirmed']),
        'Nuevos fallecidos: '+str(json_region['today_new_deaths']),
        'Nuevos recuperados: '+str(json_region['today_new_recovered'])]+ sevilla
        if region == 'Andalucía':
            sevilla = getSevilla(json_region)
            result = result + sevilla
        return result  
    else:
        return None

def getSevilla(json_region):
    for value in json_region['sub_regions']:
        if value['id'] in ['sevilla']:
            return ["Afectados en Sevilla:","Nuevos infectados: "+str(value["today_new_confirmed"]),
                    "Nuevos hospitalizados: "+str(value["today_new_total_hospitalised_patients"]),
                    "Nuevos UCI: "+str(value["today_new_intensive_care"]),
                    "Nuevos fallecidos:"+str(value['today_new_deaths']),
                   "Nuevos recuperados:"+str(value['today_new_recovered'])]

def getRegions():
    final_result=''
    regions = ['Andalucía','Galicia','Madrid','Cataluña','Castilla y León']
    for region in regions:
        results=getRegion(region)
        if results != None:
            if region == 'Andalucía':
                            final_result+=os.linesep+results[3]+os.linesep+results[4]+os.linesep+results[5]+os.linesep+results[6]+os.linesep+results[7]+os.linesep+results[8]+os.linesep
                            final_result+=os.linesep
                            final_result+='Afectados en '+region+": "+os.linesep+results[0]+os.linesep+results[1]+os.linesep+results[2]+os.linesep
                            final_result+=os.linesep
            else:   
                final_result+='Afectados en '+region+": "+os.linesep+results[0]+os.linesep+results[1]+os.linesep+results[2]+os.linesep
                final_result+=os.linesep
    return final_result
print(getRegions())



Afectados en Sevilla:
Nuevos infectados: 0
Nuevos hospitalizados: 35
Nuevos UCI: 5
Nuevos fallecidos:22
Nuevos recuperados:123

Afectados en Andalucía: 
Nuevos infectados: 1348
Nuevos fallecidos: 95
Nuevos recuperados: 60

Afectados en Galicia: 
Nuevos infectados: 291
Nuevos fallecidos: 10
Nuevos recuperados: 49

Afectados en Madrid: 
Nuevos infectados: 2072
Nuevos fallecidos: 51
Nuevos recuperados: 38

Afectados en Cataluña: 
Nuevos infectados: 2858
Nuevos fallecidos: 93
Nuevos recuperados: 31

Afectados en Castilla y León: 
Nuevos infectados: 353
Nuevos fallecidos: 12
Nuevos recuperados: None




In [57]:
now = datetime.datetime.now()
date_str = str(now.year).zfill(2)+'-'+str(now.month).zfill(2)+'-'+str(now.day-1).zfill(2)
region='Andalucia'
api_url_base = 'https://api.covid19tracking.narrativa.com/api/'+date_str+'/country/spain/region/'+region
headers = {'Content-Type': 'application/json','Authorization': 'Bearer {0}'}
response = get(api_url_base, headers=headers)
json_region = json.loads(response.content.decode('utf-8'))['dates'][date_str]['countries']['Spain']['regions'][0]['sub_regions']




In [58]:
json_region = json.loads(response.content.decode('utf-8'))['dates'][date_str]['countries']['Spain']['regions'][0]['sub_regions']

for value in json_region:
    if value['id'] in ['sevilla']:
        print("Sevilla")
        print("Nuevos casos:",value["today_new_confirmed"])
        print("Nuevos hospitalizados:",value["today_new_total_hospitalised_patients"])
        print("Nuevos UCI:",value["today_new_intensive_care"])
        print("Nuevas muertes:",value['today_new_deaths'])
        print("Nuevos recuperados:",value['today_new_recovered'])
        

Sevilla
Nuevos casos: 0
Nuevos hospitalizados: 35
Nuevos UCI: 5
Nuevas muertes: 22
Nuevos recuperados: 123


In [5]:
def ministryInformationForRegions():
    return os.linesep+pd.read_csv("https://covid19.isciii.es/resources/ccaa.csv").to_string()+os.linesep
    

In [6]:
def getDateTime():
    SEP = "/"
    now = datetime.datetime.now()
    return str(now.day).zfill(2)+SEP+str(now.month).zfill(2)+SEP+str(now.year).zfill(2)+" "+str(now.hour).zfill(2)+":"+str(now.minute).zfill(2)

In [7]:
result+=getRegions()+ministryInformationForRegions()

HTTPError: HTTP Error 503: Service Unavailable

In [ ]:
result

In [ ]:
msg = MIMEMultipart()

# setup your Gmail Parameters
password = "***"
msg['From'] = "***"
msg['To'] = "***"
msg['Subject'] = "Coronavirus, "+ getDateTime()
 
# add in the message body
result+=getRegions()+ministryInformationForRegions()
msg.attach(MIMEText(result, 'plain'))

#Gmail Server
server = smtplib.SMTP('smtp.gmail.com: 587')
 
server.starttls()
 
# Login Credentials for sending the mail
server.login(msg['From'], password)
 
 
# send the message via the server.
server.sendmail(msg['From'], msg['To'], msg.as_string())
 
server.quit()
 
print("successfully sent email to: "+msg['To'])

In [38]:
print(ministryInformationForRegions())

                  CCAA  Acumulados  Ultimas 24h  Incidencia
0            Andalucía        4682          405       50.45
1               Aragón        1858          266      129.69
2             Asturias        1088           84       92.98
3             Baleares         958           96       79.69
4             Canarias        1125          100       47.18
5            Cantabria        1023           86      167.28
6   Castilla La Mancha        5246          734      238.33
7      Castilla y León        5414          623      213.46
8             Cataluña       15026          763      186.46
9                Ceuta          21            4       23.59
10       C. Valenciana        4784          750       87.43
11         Extremadura        1456           62      127.47
12             Galicia        3139          367      109.06
13              Madrid       22677         1157      287.14
14             Melilla          48            3       46.25
15              Murcia         872      